In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata
import shutil
import gender_guesser.detector as gg

In [2]:
chambers = ['national', 'state']

In [3]:
def removeAccents(input_str):
    if isinstance(input_str, str):
        nfkd_form = unicodedata.normalize('NFKD', input_str)
        return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])
    else:
        return input_str

In [4]:
def getSlug(input_str):
    if isinstance(input_str, str):
        return removeAccents(input_str).replace(' ', '-').lower().replace('.', '').replace('/', '-')
    else:
        return input_str

In [5]:
getSlug('Appénzäll Rhôde / Intèrïeur')

'appenzall-rhode---interieur'

# 1 – récupérer les tables
### Inspect -> sauvegarder en local chaque table (générée en Angular)
* national.html https://www.parlament.ch/fr/über-das-parlament/elections2019/election-conseil-etats-2019/conseil-des-etats-a-z
* state.html https://www.parlament.ch/fr/über-das-parlament/elections2019/election-conseil-national-2019/conseil-national-a-z

Idéalement, on aurait pris des données complètes et à jour sur
https://opendata.swiss/fr/dataset/eidg-wahlen-2019, https://www.wahlen.admin.ch/data/sd-b-17.02-20191020-CH-gewaehlte.json etc…

In [6]:
docs = {}
rows = []

for chamber in chambers:
    with open(f'{chamber}.html', 'r') as f:
        doc = BeautifulSoup(f.read(), 'html.parser')
        tr = doc.select('tr')
        
        for row in tr[1:]: # skip header
            therow = [i.text.strip() for i in row.select('td')]
            img = row.select('td.profile-photo img')[0]
            href = row.find('a').get('href')
            therow.extend([img.get('src'), img.get('data-ng-alt').replace('Portrait de ', ''), href])
            therow.append(chamber)
            rows.append(therow)

In [9]:
df = pd.DataFrame(rows, columns = ['', 'nom-inverse', 'canton', 'parti', 'job', 'sortant', 'naissance', 'voix', 'image', 'name', 'href', 'chambre'])
df.shape
# yeah

(246, 12)

In [10]:
df.head()

,,nom-inverse,canton,parti,job,sortant,naissance,voix,image,name,href,chambre
0,,Addor Jean-Luc,Valais,UDC,Avocat,Sortant/E,1964,17535,/sitecollectionimages/profil/portrait-260/3055...,Jean-Luc Addor,/fr/biografie/jean-luc-addor/4154,national
1,,Aebi Andreas,Berne,UDC,Landwirt,Sortant/E,1958,104955,/sitecollectionimages/profil/portrait-260/2670...,Andreas Aebi,/fr/biografie/andreas-aebi/3867,national
2,,Aebischer Matthias,Berne,PS,,Sortant/E,1967,63397,/sitecollectionimages/profil/portrait-260/2760...,Matthias Aebischer,/fr/biografie/matthias-aebischer/4049,national
3,,Aeschi Thomas,Zoug,UDC,Unternehmensberater,Sortant/E,1979,15359,/sitecollectionimages/profil/portrait-260/2758...,Thomas Aeschi,/fr/biografie/thomas-aeschi/4053,national
4,,Amaudruz Céline,Genève,UDC,Gestionnaire de fortune,Sortant/E,1979,16434,/sitecollectionimages/profil/portrait-260/2796...,Céline Amaudruz,/fr/biografie/c%C3%A9line-amaudruz/4090,national


# 2 – récupérer et matcher les images

In [11]:
df['image_remote'] = df['image'].apply(lambda x: f'https://www.parlament.ch{x}')

In [12]:
# ou plus simple getSlug() a tester
df['filename'] = df['name'].apply(removeAccents).apply(lambda x: x.strip().replace(' ', '-').replace("'", '')).str.lower() + '.jpg'

In [13]:
with open('imglist.txt', 'w') as fp:
    fp.write( "\n".join(df['image_remote'].tolist()) )

In [14]:
# On récupère les 246 trombines officielles
# a utiliser plutot dans le term sans "quiet" (ici pour pas flooder)
# !wget -i imglist.txt -P img -q

In [15]:
df['formerpath'] = df['image_remote'].apply(lambda x: x.split('/')[-1].strip()) # oui c'est des cochons -> tout stripper

In [16]:
# On renomme
for i, row in df.iterrows():
    print('.', end='')
    shutil.move(f"img/{row['formerpath']}", f"img/{row['filename']}")

......................................................................................................................................................................................................................................................

# 3 – traitement et export json

In [17]:
for col in ['image', 'image_remote', 'formerpath']:
    del df[col]

In [18]:
df_genre = pd.read_excel('National.xls', usecols=['prénom', 'nom', 'genre'])
df_genre['name'] = df_genre['prénom'] + ' ' + df_genre['nom']

### Merge avec données OFS pour 99% du CN

In [19]:
df = df.merge(df_genre[['name', 'genre']], on='name', how='left')

In [20]:
df[df['genre'].isnull()].shape

(50, 13)

### Ajout du genre pour les 50 restants

In [21]:
d = gg.Detector()

In [22]:
df['name'] = df['name'].str.strip() # encore des foutus blancs

In [23]:
df['check-genre'] = 0
for i, row in df[df['genre'].isnull()].iterrows():
    gender = None
    if row['name'] in ['Niklaus-Samuel Gugger', 'Andrea Caroni']:
        print(row['name'], '✓')
        gender = 'male'
    elif row['name'] == 'Stéfanie Prezioso Batou':
        print(row['name'], '✓')
        gender = 'female'
    else:
        gender = d.get_gender(row['name'].split()[0])
    print(row['name'], gender)
    
    df.at[i, 'genre'] = 'F' if gender == 'female' else 'M'
    df.at[i, 'check-genre'] = 1

Mustafa Atici male
Florence Brenzikofer female
Niklaus-Samuel Gugger ✓
Niklaus-Samuel Gugger male
Lars Guggisberg male
Alfred Heer male
Stefani Heimgartner female
Piero Marchesi male
Isabelle Pasquier-Eichenberger female
Stéfanie Prezioso Batou ✓
Stéfanie Prezioso Batou female
Valentine Python female
Maja Riniker female
Heinz Siegenthaler male
Bruno Storni male
Laurent Wehrli male
Priska Wismer-Felder female
Philippe Bauer male
Elisabeth Baume-Schneider female
Pirmin Bischof male
Andrea Caroni ✓
Andrea Caroni male
Josef Dittli male
Stefan Engler male
Erich Ettlin male
Daniel Fässler male
Johanna Gapany female
Hannes Germann male
Brigitte Häberli-Koller female
Thomas Hefti male
Peter Hegglin male
Daniel Jositsch male
Charles Juillard male
Alex Kuprecht male
Christian Levrat male
Marianne Maret female
Matthias Michel male
Thomas Minder male
Damian Müller male
Ruedi Noser male
Paul Rechsteiner male
Othmar Reichmuth male
Beat Rieder male
Martin Schmid male
Carlo Sommaruga male
Jakob Stark 

In [24]:
df[df['check-genre'] == 1].to_excel('check-genre.xlsx')
!open check-genre.xlsx

### Catégorie «nouveaux» accordée au genre

In [25]:
for i, row in df[df['name'].str.contains('Fischer|Prelicz')].iterrows():
    df.at[i, 'sortant'] = 'Nouveau'

In [26]:
df['new'] = ''
for i, row in df.iterrows():
    if row['sortant'] != row['sortant']:
        continue
        
    if row['sortant'].find('Nouve') >= 0:
        if row['genre'] == 'F':
            df.at[i, 'new'] = 'Nouvelle élue'
        else:
            df.at[i, 'new'] = 'Nouvel élu'

In [27]:
# A checker: 86 nouveaux?
df[df['new'] != ''].shape

(88, 15)

In [28]:
df.groupby('chambre')['new'].value_counts()

chambre   new          
national                   133
          Nouvelle élue     37
          Nouvel élu        30
state                       25
          Nouvelle élue     11
          Nouvel élu        10
Name: new, dtype: int64

In [29]:
df['parti'].value_counts()

UDC         59
PS          48
PLR         40
PDC         37
PES         33
PVL         16
PEV          3
PBD          3
UDF          1
Lega         1
EAG          1
Autres       1
PL           1
PST/Sol.     1
             1
Name: parti, dtype: int64

In [30]:
def longParti(value):
    parti_matches = {
        'PVL': 'Vert’libéraux',
        'PEV': 'Parti évangélique',
        #'PL': 'Parti libéral démocrate',
        'PL': 'PLR',
        'PES': 'Verts',
        'EAG': 'Ensemble à gauche',
        'PST/Sol.': 'Parti du travail'
    }
    
    if value in parti_matches.keys():
        return parti_matches[value]
    else:
        return value
df['parti-long'] = df['parti'].apply(longParti)

In [31]:
df['image'] = df['filename']
del df['filename']

### Filtres

In [32]:
df['is-new'] = df['new'].apply(lambda x: 'is-new' if x != '' else '')
df['canton-filter'] = df['canton'].apply(getSlug)

In [33]:
df['new-filter'] = df['new'].apply(lambda x: 'old' if x == '' else 'new')

In [34]:
# Tri par nom de famille
df.sort_values('nom-inverse', inplace=True)

In [35]:
# Emploi indiqué
def shortenJob(value):
    if value != value:
        return ''
    elif len(value) < 45:
        return value
    else:
        return value[:45] + '…'
    
df['job-short'] = df['job'].apply(shortenJob)

In [36]:
df.tail()

,,nom-inverse,canton,parti,job,sortant,naissance,voix,name,href,chambre,genre,check-genre,new,parti-long,image,is-new,canton-filter,new-filter,job-short
199,,Zuberbühler David,Appenzell Rh.-Ext.,UDC,Unternehmer,Sortant/E,1979,7720,David Zuberbühler,/fr/biografie/david-zuberb%C3%BChler/4179,national,M,0,,UDC,david-zuberbuhler.jpg,,appenzell-rh-ext,old,Unternehmer
37,,de Courten Thomas,Bâle-Campagne,UDC,Selbständiger Unternehmer,Sortant/E,1966,24498,Thomas de Courten,/fr/biografie/thomas-de-courten/4051,national,M,0,,UDC,thomas-de-courten.jpg,,bale-campagne,old,Selbständiger Unternehmer
40,,de Quattro Jacqueline,Vaud,PLR,,Nouveau/Nouvelle,1960,44996,Jacqueline de Quattro,http://www.dequattro.ch/,national,F,0,Nouvelle élue,PLR,jacqueline-de-quattro.jpg,is-new,vaud,new,
38,,de la Reussille Denis,Neuchâtel,PST/Sol.,,Sortant/E,1960,9761,Denis de la Reussille,/fr/biografie/denis-de-la-reussille/4201,national,M,0,,Parti du travail,denis-de-la-reussille.jpg,,neuchatel,old,
186,,von Siebenthal Erich,Berne,UDC,Bergbauer,Sortant/E,1958,101660,Erich von Siebenthal,/fr/biografie/erich-von-siebenthal/3910,national,M,0,,UDC,erich-von-siebenthal.jpg,,berne,old,Bergbauer


In [37]:
df['name-search'] = df['name'].apply(removeAccents).str.replace(' ', '+')

In [38]:
df['naissance'] = df['naissance'].astype(int)

In [39]:
def getAgeCat(birthyear):
    if birthyear >= 1970 and birthyear < 1980:
        return '_1970s'
    elif birthyear >= 1980 and birthyear < 1990:
        return '_1980s'
    elif birthyear >= 1990 and birthyear < 2000:
        return '_1990s'
    elif birthyear < 1970:
        return 'pre-1970'
    else:
        print('Error with', birthyear)
df['age-filter'] = df['naissance'].apply(getAgeCat)

In [40]:
df['age-filter'].value_counts()

pre-1970    139
_1970s       57
_1980s       43
_1990s        7
Name: age-filter, dtype: int64

In [41]:
df['canton-filter'] = df['canton'].apply(getSlug)
df['parti-filter'] = df['parti'].apply(getSlug)

In [42]:
df[df['name'].str.contains('Fischer|Prelicz')]

,,nom-inverse,canton,parti,job,sortant,naissance,voix,name,href,...,new,parti-long,image,is-new,canton-filter,new-filter,job-short,name-search,age-filter,parti-filter
52,,Fischer Roland,Lucerne,PVL,,Nouveau,1965,10982,Roland Fischer,/fr/biografie/roland-fischer/4089,...,Nouvel élu,Vert’libéraux,roland-fischer.jpg,is-new,lucerne,new,,Roland+Fischer,pre-1970,pvl
139,,Prelicz-Huber Katharina,Zurich,PES,,Nouveau,1959,70499,Katharina Prelicz-Huber,/fr/biografie/katharina-prelicz-huber/3999,...,Nouvelle élue,Verts,katharina-prelicz-huber.jpg,is-new,zurich,new,,Katharina+Prelicz-Huber,pre-1970,pes


In [43]:
df['slug'] = df['name'].apply(getSlug)

In [44]:
df['chambre_tag'] = df['chambre'].apply(lambda x: 'Etats' if x == 'state' else 'National')

In [45]:
df['chambre'].value_counts()

national    200
state        46
Name: chambre, dtype: int64

In [46]:
df['job'] = df['job'].apply(lambda x: x if x != '' else 'Non indiqué')

In [47]:
df['genre'].value_counts()

M    151
F     95
Name: genre, dtype: int64

In [49]:
df.groupby('chambre')['genre'].value_counts()

chambre   genre
national  M        117
          F         83
state     M         34
          F         12
Name: genre, dtype: int64

# 4 – Export json

In [50]:
df.to_json('/Users/rnp/Sites/parlement-2019/json/parliament.json', orient='records')

## HTML

In [51]:
print(*[f'<li class="mdl-menu__item" data-filter=".{removeAccents(i)}">{longParti(i)}</li>' for i in sorted(list(set(df['parti'].value_counts().index)))], sep='\n')

<li class="mdl-menu__item" data-filter="."></li>
<li class="mdl-menu__item" data-filter=".Autres">Autres</li>
<li class="mdl-menu__item" data-filter=".EAG">Ensemble à gauche</li>
<li class="mdl-menu__item" data-filter=".Lega">Lega</li>
<li class="mdl-menu__item" data-filter=".PBD">PBD</li>
<li class="mdl-menu__item" data-filter=".PDC">PDC</li>
<li class="mdl-menu__item" data-filter=".PES">Verts</li>
<li class="mdl-menu__item" data-filter=".PEV">Parti évangélique</li>
<li class="mdl-menu__item" data-filter=".PL">PLR</li>
<li class="mdl-menu__item" data-filter=".PLR">PLR</li>
<li class="mdl-menu__item" data-filter=".PS">PS</li>
<li class="mdl-menu__item" data-filter=".PST/Sol.">Parti du travail</li>
<li class="mdl-menu__item" data-filter=".PVL">Vert’libéraux</li>
<li class="mdl-menu__item" data-filter=".UDC">UDC</li>
<li class="mdl-menu__item" data-filter=".UDF">UDF</li>


In [53]:
for i in df['age-filter'].value_counts().index:
    print(f'<li class="mdl-menu__item" data-filter=".{i}">{i}</li>')

<li class="mdl-menu__item" data-filter=".pre-1970">pre-1970</li>
<li class="mdl-menu__item" data-filter="._1970s">_1970s</li>
<li class="mdl-menu__item" data-filter="._1980s">_1980s</li>
<li class="mdl-menu__item" data-filter="._1990s">_1990s</li>
